# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.tail(5)

,Treinamento,Relevância:,Treinamento Limpo
295,discord https://t.co/q6oxummoxf\nsiga nossas m...,0,discord https//t.co/q6oxummoxfsiga nossas midi...
296,se tivesse a ps5 despedia me da marta e dos me...,0,se tivesse a ps5 despedia me da marta e dos me...
297,"desculpa pai, mas ps5 vai ter que vir no final...",0,desculpa pai mas ps5 vai ter que vir no final ...
298,"eu nem tô assim, quem tá são vocês\n\n#ps5show...",0,eu nem tô assim quem tá são vocês#ps5showcase ...
299,as vezes dá medo de pensar que o #ps5 chegue n...,1,as vezes dá medo de pensar que o #ps5 chegue n...


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1.0
1,hoje tem \n\n#ps5,0.0
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0.0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1.0
4,"@1playergirl bom, nesse momento me sinto num h...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        w = w.casefold()
        ls.append(w)
    dataframe['Treinamento Limpo'] = ls
    return dataframe
train = limpa_tweets(train)


In [6]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:
Precisamos de:
P(R|T) = P(T|R) * P(R)/ P(T)




In [6]:
#Calculando Probabilidades:
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)

    for s in dataframe['Treinamento Limpo']:
        lsWords = pd.Series(s.split())
        for i in range(len(lsWords)):
            if dataframe['Relevância:'][i+1] == 1:
                Rele = Rele.append(lsWords)
            else:
                Irrele = Irrele.append(lsWords)
    return [Rele, Irrele]
    



In [9]:
Rele = treinamento(train)[0]
Irrele = treinamento(train)[1]
total_portugues = 400000 #Consideramos todas as palavras do portugues para maximizar o smoothing.
#Calcaulando probabilidade em tweets relevantes:
lsRele = []
for v in Rele.value_counts():
    lsRele.append((v+1)/(total_portugues+Rele.count()))
lsKeys = Rele.value_counts().index.tolist()
dic_rele = {lsKeys[i]: lsRele[i] for i in range(len(lsKeys))} 
print(dic_rele)






{'o': 0.007139788506182036, 'e': 0.0057504562133699096, 'ps5': 0.005206547400864736, 'de': 0.005062687461325325, 'que': 0.004936563678715429, 'do': 0.004885325892030159, 'a': 0.004664609272462842, 'um': 0.003147182512937541, 'é': 0.002825961004102964, 'no': 0.002725456114835704, 'dia': 0.002670276959943875, 'na': 0.0020475407833075174, 'bom': 0.0019825082078992902, 'com': 0.0018879153709418688, 'vai': 0.0017381433790926181, 'da': 0.0017243485903696608, 'hoje': 0.0016967590129237464, 'não': 0.001663257383167993, 'pra': 0.001612019596482723, 'eu': 0.0015824593349335289, 'mais': 0.0015233388118351405, 'evento': 0.001332182453817018, 'sony': 0.0013164169809907812, 'uma': 0.0012888274035448666, 'preço': 0.0011469381481087344, '#ps5': 0.0011173778865595403, 'para': 0.0010286971019119577, 'tem': 0.001026726417808678, 'me': 0.0009892834198463654, 'em': 0.0009833713675365266, 'se': 0.000920309476231579, 'q': 0.0008710423736495887, 'xbox': 0.00081389253465448, 'os': 0.0008119218505512004, 'por':

In [10]:
#Calculando Probabilidade em tweets irrelevantes:
lsIrrele = []
for v in Irrele.value_counts():
    lsIrrele.append((v+1)/(total_portugues+Irrele.count()))
lsKeys = Irrele.value_counts().index.tolist()
dic_irrele = {lsKeys[i]: lsIrrele[i] for i in range(len(lsKeys))}
print(dic_irrele)

{'o': 0.006943591247587379, 'e': 0.005784344421149418, 'de': 0.005055108810672205, 'que': 0.004817314589864417, 'do': 0.004700399097967256, 'ps5': 0.004640950542765309, 'a': 0.004601318172630678, 'um': 0.0031230307666089353, 'é': 0.002740578394809745, 'dia': 0.002724725446755892, 'no': 0.0027049092616885768, 'na': 0.0020846626690815993, 'bom': 0.0020509751544671626, 'com': 0.001892445673928638, 'da': 0.001708155152802603, 'vai': 0.0016804124937083611, 'hoje': 0.0016110558459727568, 'não': 0.0015991661349323673, 'pra': 0.0015535889092775416, 'eu': 0.0015397175797304207, 'mais': 0.0014862138800486686, 'sony': 0.0012761623183351234, 'uma': 0.0012761623183351234, 'evento': 0.0012662542258014657, 'preço': 0.0010819637046754307, '#ps5': 0.0010462945715542626, 'para': 0.0010205335309667525, 'me': 0.0010007173458994368, 'em': 0.0009749563053119266, 'tem': 0.000972974686805195, 'se': 0.0008937099465359327, 'q': 0.0008441694838676437, 'os': 0.0008184084432801335, 'por': 0.0008065187322397441, 'i

In [ ]:
def classifica(dataframe, dic_rele, dic_irrele):
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**